In [13]:
# =========================================================
# 1. 라이브러리 임포트
# =========================================================
import os, time, re
import pandas as pd
from dotenv import load_dotenv
from tqdm.auto import tqdm

from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
from collections import defaultdict

In [ ]:
# 2. CSV load (인코딩은 환경에 맞게 조정: 보통 'utf-8-sig' 또는 'cp949')
# df = pd.read_csv("./dataset/perfume.csv", encoding="utf-8-sig")  # 필요시 cp949로 변경
csv_path = "./perfume.csv"
df = pd.read_csv(csv_path, encoding="utf-8-sig")  # 필요시 cp949로 변경

In [ ]:
# 3. 기본정보출력
print("\n[데이터프레임 구조]")
print(df.info())
df


[데이터프레임 구조]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1242 entries, 0 to 1241
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   brand        1240 non-null   object 
 1   name         1240 non-null   object 
 2   size_ml      1240 non-null   object 
 3   price_krw    1240 non-null   float64
 4   detail_url   1242 non-null   object 
 5   description  1240 non-null   object 
dtypes: float64(1), object(5)
memory usage: 58.3+ KB
None


,brand,name,size_ml,price_krw,detail_url,description
0,크리드,어벤투스 오 드 퍼퓸,50ml,255000.0,https://www.bysuco.com/product/show/9370,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 프루티 / 스위트 / 레더...
1,크리드,어벤투스 오 드 퍼퓸,100ml,399220.0,https://www.bysuco.com/product/show/9370,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 프루티 / 스위트 / 레더...
2,톰 포드,오드 우드 오 드 퍼퓸,30ml,179000.0,https://www.bysuco.com/product/show/10716,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 우디 / 오우드 / 웜 스...
3,톰 포드,오드 우드 오 드 퍼퓸,50ml,249000.0,https://www.bysuco.com/product/show/10716,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 우디 / 오우드 / 웜 스...
4,이솝,테싯 오 드 퍼퓸,50ml,135000.0,https://www.bysuco.com/product/show/9970,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 시트러스 / 아로마틱 / ...
...,...,...,...,...,...,...
1237,에스티 로더,알리아지 스포츠 오 드 퍼퓸,50ml,105000.0,https://www.bysuco.com/product/show/96925,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 얼씨 / 모시 / 우디\n...
1238,디올,쟈도르 롤러 펄 인피니심 오 드 퍼퓸,20ml,89000.0,https://www.bysuco.com/product/show/225110,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 화이트 플로랄 / 튜베로즈...
1239,조르지오 아르마니,씨 오 드 퍼퓸 리필,100ml,130000.0,https://www.bysuco.com/product/show/693934,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 우디 /바닐라 / 아로마틱...
1240,디올,디올 에센스 오 드 뚜왈렛,100ml,186820.0,https://www.bysuco.com/product/show/225083,[부향률] \n- 오 드 뚜왈렛\n\n[메인 어코드]\n- 웜 스파이시 / 패츌리 ...


In [22]:
# ----------------------------------------------------------------------------------------------------------------------------------------------------------------------
# 4. 데이터 전처리 
# ----------------------------------------------------------------------------------------------------------------------------------------------------------------------
#  원본 데이터 (원본 컬럼) 
#   {description} : {[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 프루티 / 스위트 / 레더\n\n[메인 노트]\n- 탑 노트: 베르가못, 블랙 커런트, 애플, 레몬, 핑크 페퍼\n- ....}
#  전처리 후 : 
#   {부향률} : {오 드 퍼퓸}
#   {메인 어코드 : {프루티, 스위트, 레더}		파인애플, 패출리, 모로칸 자스민	자작나무, 머스크, 오크 모스, 암브록산, 시더우드	용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향}
#   {탑 노트} : {베르가못, 블랙 커런트, 애플, 레몬, 핑크 페퍼}
#   {미들 노트} : {파인애플, 패출리, 모로칸 자스민}
# 	{베이스 노트} : {자작나무, 머스크, 오크 모스, 암브록산, 시더우드}
# 	{향 설명} : {용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향}
# ----------------------------------------------------------------------------------------------------------------------------------------------------------------------
#
# 4-1. parse_description(text) : description 컬럼을 부분 추출하여 별도의 컬럼 추가 생성
#   
#   1) _normalize_newlines(text) : description 문자열의 줄바꿈을 정규화(\n)
#   
#   2) [부향률] 
#       2-1) _extract_section(text, r"\[부향률\]") : 부향률 섹션 추출
#       2-2) _cleanup_bullets_and_whitespace() : 불릿(-, *), 공백을 제거 후 깔끔한 형태의 텍스트로 추출 & 컬럼 값 => ex) 부향율 : 오 드 퍼퓸
#
#   3) [메인 어코드]
#       3-1) _extract_section(text, r"\[메인\s*어코드\]") : 메인 어코드 섹션 추출
#       3-2) _cleanup_bullets_and_whitespace() : 불릿(-, *), 공백을 제거 후 깔끔한 형태의 텍스트로 추출 & 컬럼 값 -> ex) 부향율 : 오 드 퍼퓸
#       3-3) _normalize_list_string() : 구분자 변환 ( / -> , ) => ex) '프루티 / 스위트 / 레더' -> '프루티, 스위트, 레더' 형태로 통일.
#
#   4) [메인 노트]
#       4-1) _extract_section(text, r"\[메인\s*노트\]") : 메인 노트 섹션 추출 + 탑/미들/베이스 노트 섹션으로 나누기
#           4-1-1) _extract_note_line(notes_block, "탑 노트")
#           4-1-2) _extract_note_line(notes_block, "미들 노트")
#           4-1-3) _extract_note_line(notes_block, "베이스 노트")
#
#   5) [향 설명] 
#       5-1) _extract_section(text, r"\[향\s*설명\]", take_to_end=True) : 향 설명 섹션 추출
#       5-2) _cleanup_bullets_and_whitespace() : 불릿(-, *), 공백을 제거 후 깔끔한 형태의 텍스트로 추출 & 컬럼 값
#   
#   6) "description" 컬럼 -> 최종 out 딕셔너리 반환 ("부향률", "메인어코드", "메인노트", "향 설명")
# ----------------------------------------------------------------------------------------------------------------------------------------------------------------------


# ---------------------------------------------------------------------
# 유틸
# ---------------------------------------------------------------------
def _normalize_newlines(s: str) -> str:
    # \r\n, \r -> \n 통일
    # 리터럴 "\n" 도 실제 개행으로 변환
    return s.replace("\r\n", "\n").replace("\r", "\n").replace("\\n", "\n")

def _cleanup_bullets_and_whitespace(s: str) -> str:
    """
    각 줄 맨앞의 '-', '*' 같은 불릿 제거 후 줄을 공백으로 합쳐 한 줄로 만듭니다.
    """
    lines = []
    for line in s.split("\n"):
        line = re.sub(r"^\s*[-*]\s*", "", line)  # 앞의 불릿 제거
        if line.strip():
            lines.append(line.strip())
    one_line = " ".join(lines)
    one_line = re.sub(r"\s+", " ", one_line).strip()
    return one_line

def _normalize_list_string(s: str) -> str:
    """
    '프루티 / 스위트 / 레더' 같은 문자열을
    '프루티, 스위트, 레더' 형태로 통일.
    """
    # 슬래시, 쉼표(전각 포함) 등으로 분리
    parts = [p.strip() for p in re.split(r"[\/,、\u3001]+", s) if p.strip()]
    return ", ".join(parts)

def _extract_section(text: str, pattern: str, take_to_end=False) -> str:
    """
    [헤더]로 시작하는 섹션을 캡처합니다.
    - take_to_end=True 이면 문서 끝까지, False면 다음 [헤더] 직전까지.
    pattern: 예) r"\[부향률\]"
    """
    if take_to_end:
        m = re.search(pattern + r"\s*(.+)\Z", text, flags=re.S)
    else:
        m = re.search(pattern + r"\s*(.+?)(?=\n\[[^\]]+\]|\Z)", text, flags=re.S)
    return m.group(1).strip() if m else ""

def _extract_note_line(block: str, label_kr: str) -> str:
    """
    메인 노트 블록에서 '- 탑 노트: ...' 처럼 특정 라벨 줄만 뽑아 정리.
    반환은 'A, B, C' 형태 문자열.
    """
    # 줄 단위 탐색 (불릿/공백 허용, 콜론 형태 허용)
    mm = re.search(
        rf"(?m)^\s*-\s*{label_kr}\s*노트\s*[:：]\s*(.+?)\s*$",
        block
    )
    if not mm:
        return ""
    raw = mm.group(1).strip()
    return _normalize_list_string(raw)

# ---------------------------------------------------------------------
# 메인 파서
# ---------------------------------------------------------------------
def parse_description(text):
    if not isinstance(text, str):
        text = ""
    text = _normalize_newlines(text)

    out = {
        "부향률": "",
        "메인 어코드": "",
        "탑 노트": "",
        "미들 노트": "",
        "베이스 노트": "",
        "향 설명": "",
    }

    # [부향률] : 다음 [헤더] 전까지
    sec = _extract_section(text, r"\[부향률\]")
    if sec:
        out["부향률"] = _cleanup_bullets_and_whitespace(sec)

    # [메인 어코드] : 다음 [헤더] 전까지 -> 리스트 정규화 후 "A, B, C"
    sec = _extract_section(text, r"\[메인\s*어코드\]")
    if sec:
        sec = _cleanup_bullets_and_whitespace(sec)
        out["메인 어코드"] = _normalize_list_string(sec)

    # [메인 노트] 블록
    notes_block = _extract_section(text, r"\[메인\s*노트\]")
    if notes_block:
        out["탑 노트"]    = _extract_note_line(notes_block, "탑")
        out["미들 노트"]  = _extract_note_line(notes_block, "미들")
        out["베이스 노트"] = _extract_note_line(notes_block, "베이스")

    # [향 설명] : 문서 끝까지 포함 (하단 * 안내문까지 포함)
    sec = _extract_section(text, r"\[향\s*설명\]", take_to_end=True)
    if sec:
        out["향 설명"] = _cleanup_bullets_and_whitespace(sec)

    return out

# ---------------------------------------------------------------------
# 실행
# ---------------------------------------------------------------------
parsed = df["description"].apply(parse_description) # "description" 컬럼의 각 행에 parse_description() 함수 적용 -> 딕셔너리 반환
parsed_df = pd.DataFrame(parsed.tolist())           # (딕셔너리) 파싱결과를 리스트로 변환

# 원본과 합치고 저장 (Excel 호환을 위해 utf-8-sig 권장)
final_df = pd.concat([df, parsed_df], axis=1)
final_df.to_csv("after_prepro_perfume.csv", index=False, encoding="utf-8-sig")

print(final_df[["부향률", "메인 어코드", "탑 노트", "미들 노트", "베이스 노트", "향 설명"]].head(3))


      부향률           메인 어코드                         탑 노트               미들 노트  \
0  오 드 퍼퓸     프루티, 스위트, 레더  베르가못, 블랙 커런트, 애플, 레몬, 핑크 페퍼  파인애플, 패출리, 모로칸 자스민   
1  오 드 퍼퓸     프루티, 스위트, 레더  베르가못, 블랙 커런트, 애플, 레몬, 핑크 페퍼  파인애플, 패출리, 모로칸 자스민   
2  오 드 퍼퓸  우디, 오우드, 웜 스파이시                                                    

                         베이스 노트  \
0  자작나무, 머스크, 오크 모스, 암브록산, 시더우드   
1  자작나무, 머스크, 오크 모스, 암브록산, 시더우드   
2                                 

                                                향 설명  
0  용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향 일부 상품에 한하여 신규로 리...  
1  용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향 일부 상품에 한하여 신규로 리...  
2             청량한 소나무 계열의 향과 부드러운 침구가 부드럽게 감싸주는 듯한 향  


In [25]:
# "향 설명" 컬럼 
pd.set_option('display.max_colwidth', 200)
display(final_df[["향 설명"]].head(3))

,향 설명
0,"용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향 일부 상품에 한하여 신규로 리뉴얼 된 디자인의 제품이 발송될 수 있습니다. 이로 인한 교환/반품은 어려운 점 참고 부탁드립니다."
1,"용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향 일부 상품에 한하여 신규로 리뉴얼 된 디자인의 제품이 발송될 수 있습니다. 이로 인한 교환/반품은 어려운 점 참고 부탁드립니다."
2,청량한 소나무 계열의 향과 부드러운 침구가 부드럽게 감싸주는 듯한 향


In [ ]:
# 공지 문구 제거
# NOTICE_TEXT = "일부 상품에 한하여 신규로 리뉴얼 된 디자인의 제품이 발송될 수 있습니다. 이로 인한 교환/반품은 어려운 점 참고 부탁드립니다."
NOTICE_TEXT = "본 제품은 아시아에서 유통되는 상품이 아닌, 유럽에서 유통되는 상품입니다."

def clean_notice(desc: str) -> str:
    """향 설명에서 공지 문구 제거"""
    if not isinstance(desc, str):
        return desc
    # 지정된 공지 문구 제거
    cleaned = desc.replace(NOTICE_TEXT, "")
    # 불필요한 공백 정리
    return cleaned

# 적용
final_df["향 설명"] = final_df["향 설명"].apply(clean_notice)

# 저장
final_df.to_csv("after_prepro_del_txt.csv", index=False, encoding="utf-8-sig")

# "향 설명" 컬럼 
pd.set_option('display.max_colwidth', 200)
display(final_df[["향 설명"]].head(3))

,향 설명
0,"용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향"
1,"용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향"
2,청량한 소나무 계열의 향과 부드러운 침구가 부드럽게 감싸주는 듯한 향


In [30]:
import pandas as pd

df = pd.read_csv("after_prepro_del_txt.csv", encoding="utf-8-sig")
df.head()

,brand,name,size_ml,price_krw,detail_url,description,부향률,메인 어코드,탑 노트,미들 노트,베이스 노트,향 설명
0,크리드,어벤투스 오 드 퍼퓸,50ml,255000.0,https://www.bysuco.com/product/show/9370,"[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 프루티 / 스위트 / 레더\n\n[메인 노트]\n- 탑 노트: 베르가못, 블랙 커런트, 애플, 레몬, 핑크 페퍼\n- 미들 노트: 파인애플, 패출리, 모로칸 자스민\n- 베이스 노트: 자작나무, 머스크, 오크 모스, 암브록산, 시더우드\n\n[향 설명]\n- 용기와 힘, 비전, 그리고 성공을 ...",오 드 퍼퓸,"프루티, 스위트, 레더","베르가못, 블랙 커런트, 애플, 레몬, 핑크 페퍼","파인애플, 패출리, 모로칸 자스민","자작나무, 머스크, 오크 모스, 암브록산, 시더우드","용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향"
1,크리드,어벤투스 오 드 퍼퓸,100ml,399220.0,https://www.bysuco.com/product/show/9370,"[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 프루티 / 스위트 / 레더\n\n[메인 노트]\n- 탑 노트: 베르가못, 블랙 커런트, 애플, 레몬, 핑크 페퍼\n- 미들 노트: 파인애플, 패출리, 모로칸 자스민\n- 베이스 노트: 자작나무, 머스크, 오크 모스, 암브록산, 시더우드\n\n[향 설명]\n- 용기와 힘, 비전, 그리고 성공을 ...",오 드 퍼퓸,"프루티, 스위트, 레더","베르가못, 블랙 커런트, 애플, 레몬, 핑크 페퍼","파인애플, 패출리, 모로칸 자스민","자작나무, 머스크, 오크 모스, 암브록산, 시더우드","용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향"
2,톰 포드,오드 우드 오 드 퍼퓸,30ml,179000.0,https://www.bysuco.com/product/show/10716,"[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 우디 / 오우드 / 웜 스파이시\n\n[메인 노트]\n- 싱글 노트: 아가우드, 브라질리안 로즈우드, 샌달우드, 카다멈, 바닐라, 사천 페퍼, 베티버, 통카 빈, 앰버\n\n[향 설명]\n- 청량한 소나무 계열의 향과 부드러운 침구가 부드럽게 감싸주는 듯한 향",오 드 퍼퓸,"우디, 오우드, 웜 스파이시",NaN,NaN,NaN,청량한 소나무 계열의 향과 부드러운 침구가 부드럽게 감싸주는 듯한 향
3,톰 포드,오드 우드 오 드 퍼퓸,50ml,249000.0,https://www.bysuco.com/product/show/10716,"[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 우디 / 오우드 / 웜 스파이시\n\n[메인 노트]\n- 싱글 노트: 아가우드, 브라질리안 로즈우드, 샌달우드, 카다멈, 바닐라, 사천 페퍼, 베티버, 통카 빈, 앰버\n\n[향 설명]\n- 청량한 소나무 계열의 향과 부드러운 침구가 부드럽게 감싸주는 듯한 향",오 드 퍼퓸,"우디, 오우드, 웜 스파이시",NaN,NaN,NaN,청량한 소나무 계열의 향과 부드러운 침구가 부드럽게 감싸주는 듯한 향
4,이솝,테싯 오 드 퍼퓸,50ml,135000.0,https://www.bysuco.com/product/show/9970,"[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 시트러스 / 아로마틱 / 프레쉬 스파이시\n\n[메인 노트]\n- 탑 노트: 유자, 시트러스\n- 미들 노트: 바질\n- 베이스 노트: 베티버, 클로브\n\n[향 설명]\n- 이솝의 시그니처 향기로 따뜻하고 생기넘치며 마음을 릴렉싱 시켜주는 향",오 드 퍼퓸,"시트러스, 아로마틱, 프레쉬 스파이시","유자, 시트러스",바질,"베티버, 클로브",이솝의 시그니처 향기로 따뜻하고 생기넘치며 마음을 릴렉싱 시켜주는 향


In [2]:
# -----------------------------
# 2. 환경 변수 로드
# -----------------------------
load_dotenv(override=True)                 # .env 파일에서 환경 변수 로드

def getenv_clean(k: str) -> str:           # 환경 변수에서 값 가져오기
    v = os.getenv(k, "") or ""             # 기본값은 빈 문자열
    return v.strip().strip('"').strip("'") # 양쪽 따옴표 제거

OPENAI_API_KEY         = getenv_clean("OPENAI_API_KEY")
PINECONE_API_KEY       = getenv_clean("PINECONE_API_KEY")
OPENAI_EMBEDDING_MODEL = getenv_clean("OPENAI_EMBEDDING_MODEL") or "text-embedding-3-small"

CSV_PATH   = "separated_perfume_cleaned.csv"   # CSV 경로
INDEX_NAME = "perfume-search-score-0819"                   # Pinecone 인덱스명
CLOUD, REGION = "aws", "us-east-1"
EMBED_DIM  = 1536  # text-embedding-3-small

print("[DEBUG] OPENAI:", (OPENAI_API_KEY[:6]+"...") if OPENAI_API_KEY else "(empty)")
print("[DEBUG] PINECONE:", (PINECONE_API_KEY[:6]+"...") if PINECONE_API_KEY else "(empty)")

[DEBUG] OPENAI: sk-pro...
[DEBUG] PINECONE: pcsk_6...


In [8]:
# -----------------------------
# 3. 인덱스 생성
# -----------------------------
pc = Pinecone(api_key=PINECONE_API_KEY) # Pinecone 클라이언트 초기화

def ensure_index(name=INDEX_NAME, dim=EMBED_DIM, cloud=CLOUD, region=REGION): 
    
    names = [x["name"] for x in pc.list_indexes()]                            # 현재 존재하는 인덱스 목록 가져오기
    
    if name not in names:                                                     # 인덱스가 존재하지 않으면 생성
        print(f"[INFO] Creating Pinecone index '{name}' ...")                 
        pc.create_index(                                                      # 인덱스 생성
            name=name,
            dimension=dim,
            metric="cosine",
            spec=ServerlessSpec(cloud=cloud, region=region)                   # Specify cloud and region (aws, us-east-1)
        )

        while not pc.describe_index(name).status.get("ready"): # 인덱스가 준비될 때까지 대기
            time.sleep(0.5)

    idx = pc.Index(name)                # 인덱스 객체 가져오기
    _ = idx.describe_index_stats()      # 인덱스 상태 확인

    print(f"[OK] Index ready: {name}") # 인덱스 준비 완료 메시지 출력

    return idx                         # 인덱스 객체 반환

index = ensure_index()                 # 인덱스 생성함수 호출

[OK] Index ready: perfume-search-score-0819


In [ ]:
# -----------------------------
# 2) CSV → Document 변환 
# -----------------------------
# CSV 데이터를 벡터 데이터베이스(Pinecone)에 저장하기 위해서는 표 형식의 데이터를 자연어 텍스트 기반 구조(Document)로 변환필요
# LangChain, Pinecone 등에서 사용하는 Document 구조는 다음 두 가지로 구성
# 이 구조로 바꿔야 RAG 검색 시 자연어 문서로 임베딩하고 필터 조건도 함께 사용할 수 있다.

# 특정 컬럼만 page_content로 사용하고, 지정된 일부 컬럼만 metadata로 구성하여
# LangChain/Pinecone 등의 벡터 DB에 저장할 Document 객체 리스트를 생성합니다.
# -----------------------------

def csv_to_documents_selected_columns(csv_path: str) -> list[Document]:

    df = pd.read_csv(csv_path).fillna("blank")  # NaN → "blank"로 대체
    docs = []                                   # 결과 문서 리스트

    # 전체 컬럼
    # DataFrame의 전체 컬럼명을 리스트 형태로 추출
    # page_content에 모든 컬럼을 포함시키기 위해 사용
    all_cols = df.columns.tolist()

    # metadata (검색 시 필터조건으로 활용)
    metadata_cols = ["name", "brand", "price_krw", "size_ml", "메인 어코드"]

    # DataFrame의 각 행(row)을 순회
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Build Documents", unit="row"):
        
        parts = [f"{col}: {str(row[col])}" for col in all_cols if str(row[col]).strip()] # 전체 컬럼을 "컬럼명: 값" 형태로 변환해 리스트(parts)에 저장
        page_content = " | ".join(parts) if parts else " "                               # parts 리스트를 " | " 구분자로 연결하여 하나의 문자열로 만듦
        
        #metadata
        # 미리 지정한 metadata_cols만 추출해 딕셔너리(metadata)로 구성
        # 예: { "name": "향수명", "brand": "브랜드", "price_krw": "50000", ... }
        metadata = {col: str(row[col]) for col in metadata_cols}

        #Document 생성
        # LangChain의 Document 객체 생성
        # page_content: 검색·임베딩 대상 본문
        # metadata: 필터링·부가정보용 딕셔너리
        # 생성한 Document를 docs 리스트에 추가
        docs.append(Document(page_content=page_content, metadata=metadata))

    print(f"[INFO] Built {len(docs)} Documents (selected columns only).")
    return docs


# print(docs[0])
# print(docs[1])
# print(docs[2])

page_content='brand: 크리드 | name: 어벤투스 오 드 퍼퓸 | size_ml: 50ml | price_krw: 255000.0 | detail_url: https://www.bysuco.com/product/show/9370 | description: [부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 프루티 / 스위트 / 레더\n\n[메인 노트]\n- 탑 노트: 베르가못, 블랙 커런트, 애플, 레몬, 핑크 페퍼\n- 미들 노트: 파인애플, 패출리, 모로칸 자스민\n- 베이스 노트: 자작나무, 머스크, 오크 모스, 암브록산, 시더우드\n\n[향 설명]\n- 용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향\n\n*일부 상품에 한하여 신규로 리뉴얼 된 디자인의 제품이 발송될 수 있습니다.\n이로 인한 교환/반품은 어려운 점 참고 부탁드립니다. | 부향률: 오 드 퍼퓸 | 메인 어코드: 프루티, 스위트, 레더 | 탑 노트: 베르가못, 블랙 커런트, 애플, 레몬, 핑크 페퍼 | 미들 노트: 파인애플, 패출리, 모로칸 자스민 | 베이스 노트: 자작나무, 머스크, 오크 모스, 암브록산, 시더우드 | 향 설명: 용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향' metadata={'name': '어벤투스 오 드 퍼퓸', 'brand': '크리드', 'price_krw': '255000.0', 'size_ml': '50ml', '메인 어코드': '프루티, 스위트, 레더'}
page_content='brand: 크리드 | name: 어벤투스 오 드 퍼퓸 | size_ml: 100ml | price_krw: 399220.0 | detail_url: https://www.bysuco.com/product/show/9370 | description: [부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 프루티 / 스위트 / 레더\n\n[메인 노트]\n- 탑 노트: 베르가못, 블랙 커런트, 애플, 레몬, 핑크 페퍼\n- 미들 노트: 파인애플, 

In [19]:
# -----------------------------
# 3) Pinecone에 업서트
# -----------------------------

embeddings = OpenAIEmbeddings(model=OPENAI_EMBEDDING_MODEL, api_key=OPENAI_API_KEY)
vector_store = PineconeVectorStore(index_name=INDEX_NAME, embedding=embeddings)

def upsert_documents(docs: list[Document], batch_size: int = 100, id_prefix="perfume"): 

    n = len(docs)                   # 문서 개수
    if n == 0:                      # 문서가 없으면 경고 메시지 출력
        print("[WARN] no docs")     # 문서가 없으면 업서트하지 않음
        return
    print(f"Upserting {n} docs (batch={batch_size}) ...")

    

    for i in tqdm(range(0, n, batch_size), desc="Upsert", unit="batch"):    # 배치 단위로 업서트 
        batch = docs[i:i+batch_size]                                        # 현재 배치의 문서들 (docs 리스트에서 i부터 i+batch_size 직전까지 잘라서 batch라는 리스트에 저장)
        ids = [f"{id_prefix}-{i+j}" for j in range(len(batch))]             # 각 문서의 ID 생성 (id_prefix + 인덱스)

        try: # 업서트 시도
            vector_store.add_documents(batch, ids=ids)                     # 문서 업서트
        except Exception as e: # 예외 발생 시
            print(f"[ERROR] 업서트 실패 (docs {i} ~ {i+len(batch)-1}): {e}") # 예외 메시지 출력


    stats = pc.Index(INDEX_NAME).describe_index_stats()                     # 인덱스 상태 가져오기
    print("Done. total_vector_count:", stats.get("total_vector_count"))     # 업서트 완료 메시지와 벡터 개수 출력
    return stats


In [ ]:
# -----------------------------
# 4) Field-Weighted Retriever 검색 함수
# -----------------------------
# field_queries: 검색할 컬럼과 쿼리, 가중치가 들어있는 dict
# top_k : 검색 상위 몇 개 문서 가져올지
# sep : 출력 시 구분자
# scores: 각 문서별 누적 점수 저장 (가중치 반영)
# -----------------------------


# 결과 출력 시 사용할 컬럼 헤더 순서 정의
STANDARD_COLS = ["brand", "name", "향 설명","메인 어코드", "탑 노트", "미들 노트", "베이스 노트"]

def search_with_field_weights(field_queries: dict, top_k: int = 5, sep: str = " / "):
    scores = defaultdict(float) # score를 0.0으로 초기화
    docs_by_id = {} # docs_by_id: 문서 ID → Document 객체 매핑

    # field_queries는 {컬럼명: (검색어, 가중치)} 형태
    for field, (query, weight) in field_queries.items():

        # {컬럼명: (검색어, 가중치)} => {컬럼명: 검색어} 변환
        full_query = f"{field}: {query}"          

        # vector_store.as_retriever(...) → 벡터DB를 LangChain 표준 검색 인터페이스(Retriever)로 바꿔줍니다.
        # .invoke(full_query) → full_query 문장과 의미적으로 가까운 문서 상위 top_k개를 가져옴
        docs = vector_store.as_retriever(search_kwargs={"k": top_k}).invoke(full_query)

        for doc in docs:
            doc_id = doc.metadata.get("detail_url", doc.page_content[:50]) # doc_id: 문서 고유 식별자
            scores[doc_id] += weight                                       # scores[doc_id]: 그 문서의 누적 점수 / field의 가중치(weight)만큼 더해짐

            # 아직 저장되지 않은 문서라면 docs_by_id에 등록
            # 나중에 점수 기반으로 정렬할 때 원본 doc 객체를 꺼내 쓰기 위함
            if doc_id not in docs_by_id:
                docs_by_id[doc_id] = doc

    if not docs_by_id:
        print("검색 결과 없음.")
        return pd.DataFrame()

    # docs_by_id.items() -> (doc_id, doc) 쌍들의 리스트
    # -scores[x[0]] : 점수가 큰 문서가 앞에 오도록 내림차순 정렬
    sorted_docs = sorted(docs_by_id.items(), key=lambda x: -scores[x[0]])
    

    # top_docs : 정렬된 문서 중 top_k개만 추출
    # _는 doc_id는 버리고 doc 객체만 리스트로 저장
    top_docs = [doc for _, doc in sorted_docs[:top_k]]

    # print(sep.join(STANDARD_COLS))

    rows = [] #최종적으로 DataFrame에 넣을 행(dict)들의 리스트
    
    for d in top_docs:
        meta = d.metadata or {} # 각 문서마다 metadata를 꺼냄 (없으면 {})
            
        # “page_content를 ' | ' 기준으로 나눈 뒤, ':'가 있는 항목만 골라서 key: value로 분리
        parsed_from_content = dict(
            item.split(":", 1) for item in d.page_content.split(" | ") if ":" in item
        )
        # 1) d.page_content.split(" | ") => page_content를 "|" 기준으로 나눔 / ["brand: 딥디크", "name: 도 손", "메인 어코드: woody"]
        # 2) for item in ... =>  1st "brand: 딥디크" / 2rd "name: 도 손" / 3th "메인 어코드: woody"
        # 3) if ":" in item => item 안에 :가 있는 경우만 처리
        # 4) item.split(":", 1) => "brand: 딥디크" → ["brand", " 딥디크"]
        # 5) 최종적으로 [["brand", " 딥디크"], ["name", " 도 손"], ["메인 어코드", " woody"]] 같은 리스트를 만들어냄
        
        # 6) dict로 감싸면 아래와 같은 딕셔너리 생성
        # {
        #   "brand": " 딥디크",
        #   "name": " 도 손",
        #   "메인 어코드": " woody"
        # }

        # 위의 리스트 컴프리헨션 문을 아래와 같은 for문으로 풀어 쓸 수 있다.
        # parsed_from_content = {}

        # for item in d.page_content.split(" | "):   # " | " 기준으로 나누기
        #     if ":" in item:                        # ":" 있는 경우일떄 
        #       key, value = item.split(":", 1)    # 앞: key, 뒤: value
        #       parsed_from_content[key] = value

        # key나 value 앞뒤에 불필요한 공백을 제거
        parsed_from_content = {k.strip(): v.strip() for k, v in parsed_from_content.items()}

        

        # 2. STANDARD_COLS 순서대로 값 채우기

        row_vals = [] # DataFrame 한 행에 들어갈 값들을 순서대로 넣을 리스트

        for col in STANDARD_COLS:
            val = meta.get(col) or parsed_from_content.get(col) or "" # 출력컬럼에 대한 값을 찾음

            row_vals.append(val.replace("\r\n", "\\n").replace("\n", "\\n")) 
            # val안에 들어있던 실제 줄바꿈(\r\n or \n)을 줄바꿈 텍스트(\\n)로 치환
            # DataFrame에 넣었을 때 줄이 깨지지 않고 "라벤더\n머스크"처럼 문자열로 유지
            # window 스타일 줄바꿈 : \r\n
            # Unix 스타일 줄바꿈 : \n

        # print(sep.join(row_vals))

        row_dict = dict(zip(STANDARD_COLS, row_vals))   # STANDARD_COLS와 row_vals를 묶어 딕셔너리 생성
        # row_dict["page_content"] = d.page_content       # 원문 전체 텍스트(page_content)도 함께 저장
        rows.append(row_dict)                           # 완성된 한 행(dict)을 rows 리스트에 추가

    return pd.DataFrame(rows, columns=STANDARD_COLS)

In [11]:
# =========================================================
# 사용 예시
# =========================================================
# [필요시 1회] CSV → 문서(all columns) → Pinecone 업서트
# docs = csv_to_documents_all_columns(CSV_PATH)
docs = csv_to_documents_selected_columns(CSV_PATH)
# stats = upsert_documents(docs, batch_size=100)

Build Documents:   0%|          | 0/1242 [00:00<?, ?row/s]

[INFO] Built 1242 Documents (selected columns only).


In [12]:
field_queries = {
    "메인어코드": ("우디", 1.0),
    "향설명": ("남성", 10.0),
    "브랜드명": ("딥디크", 2.0),
}

df = search_with_field_weights(field_queries, top_k=20)
display(df)

메인어코드: 우디
향설명: 남성
브랜드명: 딥디크


,brand,name,향 설명,메인 어코드,탑 노트,미들 노트,베이스 노트
0,메종 마르지엘라,재즈 클럽 오 드 뚜왈렛,프라이빗 재즈 클럽에서 느껴지는 남성적이고 활기찬 향,"타바코, 스위트, 럼","핑크페퍼, 네롤리, 레몬","럼, 자바 제티버 오일, 세이지","타바코 리프, 바닐라 빈, 때죽나무"
1,입생로랑,쿠로스 오 드 뚜왈렛,승리하는 남성성의 시대를 초월한 향,"머스크, 아로마틱, 파우더리","알데히드, 고수, 클라리 세이지, 아르테미시아, 베르가못","패츌리, 카네이션, 제라늄, 베티버, 시나몬, 자스민, 오리스 뿌리, 라벤더","시빗, 꿀, 레더, 머스키, 오크모스, 앰버, 통카 빈, 바닐라"
2,샤넬,블루 드 샤넬 퍼퓸,성취감과 자신감을 보여주는 강렬한 남성의 향기,"우디, 시트러스, 아로마틱","레몬 제스트, 베르가못, 민트, 아르테미지아","라벤더, 파인애플, 제라늄, 그린 노트","샌달우드, 시더, 앰버우드, 통카빈, 이소 이 수퍼"
3,입생로랑,르 옴므 오 드 뚜왈렛,신선한 우디 향의 매혹적이고 관능적인 우아함,"웜 스파이시, 시트러스, 아로마틱","진저, 베르가못, 레몬","향료, 바이올렛 잎, 화이트 페퍼, 바질","통카 빈, 시더, 타히티 베티버"
4,입생로랑,르 옴므 오 드 뚜왈렛,신선한 우디 향의 매혹적이고 관능적인 우아함,"웜 스파이시, 시트러스, 아로마틱","진저, 베르가못, 레몬","향료, 바이올렛 잎, 화이트 페퍼, 바질","통카 빈, 시더, 타히티 베티버"
5,샤넬,블루 드 샤넬 오 드 퍼퓸,현실과 규칙에서 벗어나 진취적으로 삶을 살아가는 남성을 위한 향,"시트러스, 앰버, 우디","그레이프 프룻, 레몬, 민트, 핑크 페퍼, 베르가못, 알데하이드, 코리안더","진저, 너트맥, 자스민, 멜론","인센스, 앰버, 시더, 샌달우드, 패출리, 앰버우드, 랍다넘"
6,샤넬,알뤼르 옴므 오 드 뚜왈렛,과감한 결단력과 카리스마를 지닌 남성을 표현한 향,"시트러스, 바닐라, 웜 스파이시","레몬, 피치, 진저, 만다린 오렌지, 라벤더, 베르가못","페퍼, 시더, 패출리, 베티버, 브라질리안 로즈우드, 로즈, 자스민, 가데니아, 아...","바닐라, 통카 빈, 샌달우드, 코코넛, 앰버, 벤조인, 머스크, 레더, 오크모스"
7,입생로랑,라 뉘 드 롬므 퍼퓸,이브 생 로랑 남성성의 새로운 측면을 소개하는 유혹의 퍼퓸,"프레쉬 스파이시, 프루티, 스위트","페퍼, 아니스, 베르가못","프루티 노트, 라벤더, 프렌치 라다넘","바닐라, 패츌리, 베티버"
8,프레데릭 말,무슈 오 드 퍼퓸,남자의 동물적인 감각을 일깨우는 강렬한 패츌리 향,"우디, 패출리, 앰버","럼, 텐저린","패출리, 인센스, 시더, 앰버","머스크, 바닐라"
9,프레데릭 말,무슈 오 드 퍼퓸,남자의 동물적인 감각을 일깨우는 강렬한 패츌리 향,"우디, 패출리, 앰버","럼, 텐저린","패출리, 인센스, 시더, 앰버","머스크, 바닐라"
